# Getting started with ODE in Julia

Based on: https://docs.sciml.ai/DiffEqDocs/stable/getting_started/ 

## Solving scalar equations

In [ ]:
using DrWatson 
@quickactivate "diff_gleam"
using DifferentialEquations
using Plots

In [ ]:
f(u,p,t) = 1.01 * u
u0 = 1/2
tspan = (0.0, 1.0)
prob = ODEProblem(f, u0, tspan)

Now not using event handlers, BUT this can become crucial later!

In [ ]:
sol = solve(prob, reltol = 1e-6, saveat = 0.1)

In [ ]:
sol.t

In [ ]:
sol.u

In [ ]:
plot(sol)

https://docs.sciml.ai/DiffEqDocs/stable/basics/common_solver_opts/#solver_options Through hints, the right solver can be chosen! e.g. `alg_hints = [:stiff]`. Tsit5() for standard non-stiff. This is the first algorithm to try in most cases.

In [ ]:
sol = solve(prob, Tsit5(), reltol = 1e-6, saveat = 0.1)

In [ ]:
sol(0.4343) #INTERPOLATES to the solution!
#this interpolation is high order if saveat is NOT used (if saveat is used, it is linear)

### Own experiment: what if the differential equation is a matrix

In [ ]:
using Pkg
Pkg.rm("ProgressLogging")

In [ ]:
f_matrix(u,p,t) = 1.01 .* u
u0_matrix = rand(1000,1000) #went to 10 000 x 10 000
matrix_problem = ODEProblem(f_matrix, u0_matrix, tspan)
sol_matrix = solve(matrix_problem, DP5(), progress = true)

In [ ]:
Array(sol_matrix)

In [ ]:
non_matrix_problem = ODEProblem(f_matrix, u0_matrix, tspan)
sol_matrix = solve(matrix_problem)

## 2 Solving systems of equations

In [ ]:
#don't forget that the ! overwrites the du!
function lorenz!(du, u, p, t)
    du[1] = 10.0*(u[2] - u[1])
    du[2] = u[1]*(28.0 - u[3]) - u[2]
    du[3] = u[1]*u[2] - (8/3) * u[3]
end 
u0 = [1.0;0.0;0.0]
tspan = (0.0, 100.0)
prob = ODEProblem(lorenz!, u0, tspan)
sol = solve(prob)
plot(sol, idxs =(1,2,3))

Can be written more genereal!

In [ ]:
function param_lorenz!(du, u, p, t)
    du[1] = p[1]*(u[2] - u[1])
    du[2] = u[1]*(p[2] - u[3]) - u[2]
    du[3] = u[1]*u[2] - (p[3]) * u[3]
end 
u0 = [1.0;0.0;0.0]
tspan = (0.0, 100.0)
p = [10.0, 28.0, 8/3]
prob = ODEProblem(param_lorenz!, u0, tspan,p)
sol = solve(prob)
plot(sol, idxs =(1,2,3))

In [ ]:
function parameterized_lorenz!(du, u, p, t)
    x, y, z = u
    σ, ρ, β = p
    du[1] = dxdt = σ * (y - x)
    du[2] = dydt = x * (ρ - z) - y
    du[3] = dzdt = x * y - β * z
end
#this looks way nicer :)

This problem can of course also be solved using the `ModellingToolkit.jl` interface!

In [ ]:
using ModelingToolkit

In [ ]:

function mtk_lorenz(;name)
    @variables t, x(t), y(t), z(t)
    D = Differential(t)
    @parameters σ, ρ, β
    eqs = [
        D(x) ~ σ*(y-x),
        D(y) ~ x*(ρ-z) - y,
        D(z) ~ x*y - β*z
    ]
    return ODESystem(eqs; name)
end
# mtk_l_test = mtk_lorenz(name = :test)
@named mtk_l_test = mtk_lorenz()
mtk_l_test = complete(mtk_l_test) #crucial to indicate that model is complete!
u0 = [mtk_l_test.x => 1.0, mtk_l_test.y => 0.0, mtk_l_test.z => 0.0]
p = [mtk_l_test.σ => 10.0, mtk_l_test.ρ => 28.0, mtk_l_test.β => 8/3]
prob = ODEProblem(mtk_l_test, u0, (0.0, 100.0), p)
sol = solve(prob)
plot(sol, idxs = (1,2,3))

In [ ]:
mtk_l_test

So modellingtoolkit the advantage of the nice symbolic representation of your model, but disadvantage of being harder to directly link to the ode solver!

## Nonhomogeneous equations with parametrised functions!

In [ ]:
l = 1.0  # lenght [m]
m = 1.0  # mass [kg]
g = 9.81 # gravitational acceleration [m/s^2]

function pendulum!(du, u, p, t) #so changes du with the !
    θ, ω = u
    M(t) = p(t)
    du[1] = dθdt = ω
    du[2] = dωdt = -3g /(2l) * sin(θ) + 3/(m*l^2)*M(t)
end
M = t -> 0.1sin(t) #anonymous function
θ₀ = 0.01                           # initial angular deflection [rad]
ω₀ = 0.0                            # initial angular velocity [rad/s]
u₀ = [θ₀, ω₀]                       # initial state vector
tspan = (0.0, 10.0)                 # time interval

prob = ODEProblem(pendulum!, u₀, tspan, M)
sol = solve(prob)
plot(sol, layout = (2,1), label = ["θ [rad]" "ω [rad/s]"])

**Note also that, in contrast with the time-varying parameter, the (vector of) state variables u, which is generally also time-varying, is always used without the explicit dependence on time (t).**

In [ ]:
prob